# NBA Single-Player Search
**Programmer:** Rahim Hashim<br>
**Date:** Sep 2021<br>
**Description:** Scrape basketball-reference.com by single players.<br>

Data captured includes —<br>
* Player Bios<br>
  * *Body Measurements* (height, weight, age...)
  * *Draft* (round, pick, year...)
  * *Hometown* 
  * *College Team*
* Season<br>
  * *Per Game* (pts/game, reb/game, assists/game...)
  * *Totals* (pts, reb, assists...)
  * *Per 36 Minutes* (pts, reb, assists...)
  * *Per 100 Possessions* (pts, reb, assists...)
  * *Advanced* (PER, OBPM, DBPM, VORP...)
  * *Play-by-Play* (PG%, SG%, And1, Blkd...)
  * *Shooting* (FG% by distance, Dunks, Corner 3s...)
* Game Logs <br>
  * Basic and Advanced stats 
  * Win-Loss
* Play-By-Play
  * Real time elapsed between shots, lineups

***

In [1]:
%reload_ext autoreload
import os
import sys
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict, defaultdict
pd.options.mode.chained_assignment = None  # default='warn'

ROOT = '/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/' #@param ['/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/']  

def add_helpers():
  '''
  add_helper mounts google drive and adds
  helper functions to the sys.path
  '''

  # if running on juypter/google colab, mount to google drive
  if 'google.colab' in str(get_ipython()): 
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    os.chdir(ROOT)

  helper_dir_path = ROOT + 'helper/'
  sys.path.append(helper_dir_path) # set to path of notebook

add_helpers()

Mounted at /content/drive


In [2]:
from player_scraper import single_player_scraper

kb_meta, kb_data, kb_gamelogs = single_player_scraper()
kb_meta

Player Name: Wilt Chamberlain
Searching for name in: https://www.basketball-reference.com/players/c
  Bill Chamberlain (match score=0.875)
  Wilt Chamberlain (match score=1.0)
Best match: Wilt Chamberlain


,birthCountry,birthCity,birthState,shootingHand,highSchool,highSchoolCity,highSchoolState,highSchoolCountry,draftTeam,draftRound,draftRoundPick,draftOverallPick
0,United States of America,Philadelphia,Pennsylvania,Right,Overbrook,Philadelphia,Pennsylvania,United States of America,Philadelphia Warriors,1,3,3


In [4]:
#@title Table Select { run: "auto" }

#@markdown Per Game | Totals | Advanced | Per Minute | Per Possession | Adjusted Shooting | Play-By-Play | Shooting | All-Star | Salaries
table_type = "advanced" #@param ["all", "per_game", "totals", "advanced", "per_minute", "per_poss", "adjooting", "pbp", "shooting", "all_star", "all_salaries"] {allow-input: true}

#@markdown Season or Playoff Stats
season_playoffs = "both" #@param ["both", "season", "playoffs"]

#@markdown Include or Exclude Career Data
career_data = "exclude" #@param ["include", "exclude"]

def table_select(df, table_type, season_playoffs):
  df_filtered = df.copy(deep=True)
  if table_type != 'all':
    df_filtered = df[df['data_type'] == table_type]
    df_filtered = df_filtered.dropna(how='all', axis='columns')
  if season_playoffs != 'both':
    df_filtered = df_filtered[df_filtered['season_playoffs'] == season_playoffs]
  if career_data == 'exclude':
    df_filtered = df_filtered[df_filtered['season'] != 'Career']

  return df_filtered

df_table = table_select(kb_data, table_type, season_playoffs)
df_table

,data_type,season_playoffs,player_name,season,age,team_id,lg_id,pos,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_48,obpm,dbpm,bpm,vorp
62,advanced,season,Wilt Chamberlain,1959-60,23,PHW,NBA,C,72,3338,28.0,0.493,,0.429,,,,,,,,,9.0,8.0,17.0,0.245,,,,
63,advanced,season,Wilt Chamberlain,1960-61,24,PHW,NBA,C,79,3773,27.8,0.519,,0.429,,,,,,,,,12.3,6.5,18.8,0.240,,,,
64,advanced,season,Wilt Chamberlain,1961-62,25,PHW,NBA,C,80,3882,31.8,0.536,,0.431,,,,,,,,,17.1,6.0,23.1,0.286,,,,
65,advanced,season,Wilt Chamberlain,1962-63,26,SFW,NBA,C,80,3806,31.8,0.550,,0.402,,,,,,,,,16.0,5.0,20.9,0.264,,,,
66,advanced,season,Wilt Chamberlain,1963-64,27,SFW,NBA,C,80,3689,31.6,0.537,,0.442,,,,,,,,,14.4,10.6,25.0,0.325,,,,
67,advanced,season,Wilt Chamberlain,1964-65,28,TOT,NBA,C,73,3301,28.6,0.513,,0.422,,,,13.8,,,,,9.6,5.5,15.1,0.219,,,,
68,advanced,season,Wilt Chamberlain,1964-65,28,SFW,NBA,C,38,1743,29.8,0.495,,0.392,,,,13.6,,,,,4.3,3.2,7.5,0.208,,,,
69,advanced,season,Wilt Chamberlain,1964-65,28,PHI,NBA,C,35,1558,27.3,0.540,,0.470,,,,14.1,,,,,5.3,2.3,7.6,0.233,,,,
70,advanced,season,Wilt Chamberlain,1965-66,29,PHI,NBA,C,79,3737,28.3,0.547,,0.490,,,,16.8,,,,,12.9,8.5,21.4,0.275,,,,
71,advanced,season,Wilt Chamberlain,1966-67,30,PHI,NBA,C,81,3682,26.5,0.637,,0.761,,,,21.7,,,,,14.8,7.0,21.9,0.285,,,,


In [6]:
df_table.sort_values(by='ws', ascending=False).head(10)

,data_type,season_playoffs,player_name,season,age,team_id,lg_id,pos,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_48,obpm,dbpm,bpm,vorp
66,advanced,season,Wilt Chamberlain,1963-64,27,SFW,NBA,C,80,3689,31.6,0.537,,0.442,,,,,,,,,14.4,10.6,25.0,0.325,,,,
64,advanced,season,Wilt Chamberlain,1961-62,25,PHW,NBA,C,80,3882,31.8,0.536,,0.431,,,,,,,,,17.1,6.0,23.1,0.286,,,,
71,advanced,season,Wilt Chamberlain,1966-67,30,PHI,NBA,C,81,3682,26.5,0.637,,0.761,,,,21.7,,,,,14.8,7.0,21.9,0.285,,,,
70,advanced,season,Wilt Chamberlain,1965-66,29,PHI,NBA,C,79,3737,28.3,0.547,,0.490,,,,16.8,,,,,12.9,8.5,21.4,0.275,,,,
65,advanced,season,Wilt Chamberlain,1962-63,26,SFW,NBA,C,80,3806,31.8,0.550,,0.402,,,,,,,,,16.0,5.0,20.9,0.264,,,,
72,advanced,season,Wilt Chamberlain,1967-68,31,PHI,NBA,C,82,3836,24.7,0.557,,0.677,,,,23.2,,,,,9.7,10.7,20.4,0.255,,,,
63,advanced,season,Wilt Chamberlain,1960-61,24,PHW,NBA,C,79,3773,27.8,0.519,,0.429,,,,,,,,,12.3,6.5,18.8,0.240,,,,
77,advanced,season,Wilt Chamberlain,1972-73,36,LAL,NBA,C,82,3542,19.1,0.689,,0.776,,,19.6,12.5,,,,,10.7,7.5,18.2,0.247,,,,
62,advanced,season,Wilt Chamberlain,1959-60,23,PHW,NBA,C,72,3338,28.0,0.493,,0.429,,,,,,,,,9.0,8.0,17.0,0.245,,,,
76,advanced,season,Wilt Chamberlain,1971-72,35,LAL,NBA,C,82,3469,18.5,0.610,,0.686,,,20.1,11.2,,,,,7.9,7.9,15.8,0.219,,,,


In [ ]:
kb_gamelogs

,player_name,season,season_playoffs,rk,g,date,age,tm,loc,opp,win,gs,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,gmsc,plus_minus,note
0,Kobe Bryant,1996-97,season,1,,1996-11-01,18-070,LAL,,PHO,W (+14),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Did Not Play
1,Kobe Bryant,1996-97,season,2,1,1996-11-03,18-072,LAL,,MIN,W (+6),0,6:22,0,1,0,0,0,,0,0,,0,1,1,0,0,1,1,1,0,-1.1,-8,
2,Kobe Bryant,1996-97,season,3,2,1996-11-05,18-074,LAL,@,NYK,W (+6),0,3:16,0,1,0,0,0,,1,2,0.5,0,0,0,0,0,0,1,0,1,-1.1,-8,
3,Kobe Bryant,1996-97,season,4,3,1996-11-06,18-075,LAL,@,CHH,L (-10),0,6:49,2,3,0.667,1,2,0.5,0,0,,0,0,0,0,0,0,3,0,5,0.7,0,
4,Kobe Bryant,1996-97,season,5,4,1996-11-08,18-077,LAL,@,TOR,L (-1),0,17:15,3,8,0.375,2,3,0.667,2,4,0.5,0,3,3,0,0,1,0,3,10,5.2,2,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,Kobe Bryant,2015-16,season,78,62,2016-04-06,37-227,LAL,,LAC,L (-10),1,27:46,6,19,0.316,1,6,0.167,4,4,1,0,3,3,1,1,0,1,1,17,7.3,-15,
78,Kobe Bryant,2015-16,season,79,63,2016-04-08,37-229,LAL,@,NOP,L (-8),1,22:13,4,15,0.267,3,9,0.333,3,3,1,0,3,3,4,1,0,2,0,14,7.8,-16,
79,Kobe Bryant,2015-16,season,80,64,2016-04-10,37-231,LAL,@,HOU,L (-20),1,27:26,10,22,0.455,4,11,0.364,11,11,1,0,2,2,1,2,0,4,1,35,22.5,-10,
80,Kobe Bryant,2015-16,season,81,65,2016-04-11,37-232,LAL,@,OKC,L (-33),1,18:57,4,12,0.333,3,9,0.333,2,2,1,0,1,1,0,1,0,2,2,13,4.7,-24,


In [ ]:
type(kb_gamelogs['gmsc'].iloc[5])

numpy.float64

[Play-By-Play Stuff](https://jman4190.medium.com/how-to-accessing-live-nba-play-by-play-data-f24e02b0a976)

In [ ]:
kb_gamelogs

,player_name,season,season_playoffs,rk,g,date,age,tm,loc,opp,win,gs,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,gmsc,plus_minus,note
0,Kobe Bryant,1996-97,season,1,,1996-11-01,18-070,LAL,,PHO,W (+14),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Did Not Play
1,Kobe Bryant,1996-97,season,2,1,1996-11-03,18-072,LAL,,MIN,W (+6),0,6:22,0,1,0,0,0,,0,0,,0,1,1,0,0,1,1,1,0,-1.1,-8,
2,Kobe Bryant,1996-97,season,3,2,1996-11-05,18-074,LAL,@,NYK,W (+6),0,3:16,0,1,0,0,0,,1,2,0.5,0,0,0,0,0,0,1,0,1,-1.1,-8,
3,Kobe Bryant,1996-97,season,4,3,1996-11-06,18-075,LAL,@,CHH,L (-10),0,6:49,2,3,0.667,1,2,0.5,0,0,,0,0,0,0,0,0,3,0,5,0.7,0,
4,Kobe Bryant,1996-97,season,5,4,1996-11-08,18-077,LAL,@,TOR,L (-1),0,17:15,3,8,0.375,2,3,0.667,2,4,0.5,0,3,3,0,0,1,0,3,10,5.2,2,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,Kobe Bryant,2015-16,season,78,62,2016-04-06,37-227,LAL,,LAC,L (-10),1,27:46,6,19,0.316,1,6,0.167,4,4,1,0,3,3,1,1,0,1,1,17,7.3,-15,
78,Kobe Bryant,2015-16,season,79,63,2016-04-08,37-229,LAL,@,NOP,L (-8),1,22:13,4,15,0.267,3,9,0.333,3,3,1,0,3,3,4,1,0,2,0,14,7.8,-16,
79,Kobe Bryant,2015-16,season,80,64,2016-04-10,37-231,LAL,@,HOU,L (-20),1,27:26,10,22,0.455,4,11,0.364,11,11,1,0,2,2,1,2,0,4,1,35,22.5,-10,
80,Kobe Bryant,2015-16,season,81,65,2016-04-11,37-232,LAL,@,OKC,L (-33),1,18:57,4,12,0.333,3,9,0.333,2,2,1,0,1,1,0,1,0,2,2,13,4.7,-24,
